In [1]:
# To hold the code of a blank transformer model
import sys
sys.path.append('/workspace/fourth_year_project/MusicGen')
#print(sys.path)

In [2]:
from MyAudioDataset import MyAudioDataset

In [3]:
import torch
from torch import nn
import torchaudio

In [9]:

class AudioTransformer(nn.Module):
    def __init__(self, source_comp, target_comp, d_model, nhead, num_layers, dim_feedforward):
        super(AudioTransformer, self).__init__()
        self.transformer = nn.Transformer(d_model, nhead, num_layers, dim_feedforward)
        self.input_encoding = nn.Linear(1500, d_model)  # input of mono audio
        #self.angle_encoding = nn.Linear(6, d_model)  # add this once 90 works. 
        self.output_decoding = nn.Linear(d_model, 2)  # Decoding back to stereo audio

        self.source_comp = source_comp
        self.target_comp = target_comp

    def forward(self, audio, angle):
        audio = audio.cuda()
        print(audio.shape)
        audio = self.input_encoding(audio)

        #angle = self.angle_encoding(angle)  # Process one-hot encoded angle
        #angle = angle.unsqueeze(1).repeat(1, audio.size(2), 1)  # Repeat angle for each time step
        #x = audio + angle  # Combine audio and angle

        x = self.transformer(x.permute(2, 0, 1))
        x = x.permute(1, 2, 0)
        x = self.output_decoding(x)
        return x.view(-1, 2, 30*44100)
    
    def compress_mono_file(self, mono_path):
        wav, sr = torchaudio.load(mono_path)
        wav = wav.cuda().unsqueeze(0)
        with torch.no_grad():
            wav, scale = self.source_comp.encode(wav)
        return wav
    
    def compress_mono(self, mono):
        mono = mono.cuda()
        with torch.no_grad():
            mono, scale = self.source_comp.encode(mono)
        return mono
    
    def compress_stereo_file(self, stereo_path):
        wav, sr = torchaudio.load(stereo_path)
        wav = wav.cuda().unsqueeze(0)
        with torch.no_grad():
            wav, scale = self.target_comp.encode(wav)
        return wav
    
    def compress_stereo(self, stereo):
        stereo = stereo.cuda()
        with torch.no_grad():
            stereo, scale = self.target_comp.encode(stereo)
        return stereo

    
    def train(self, dataset, batch_size, epochs, lr):
        #loss_fn = nn.MSELoss()
        loss_fn = nn.CosineEmbeddingLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            for i, (target, orig, angle, sr) in enumerate(train_loader):
                optimizer.zero_grad()
                target = target.cuda()
                orig = orig.cuda()
                # Convert wav to codes
                target_codes = self.compress_stereo(target)
                orig_codes = self.compress_mono(orig)
                print(type(target_codes[0][0]))
                print(type(orig_codes[0][0]))
                # Pass codes to model
                output = self(orig_codes, angle)
                loss = loss_fn(output, target_codes)
                loss.backward()
                optimizer.step()
                print(f"Epoch {epoch}, batch {i}, loss {loss.item()}")

            torch.save(self.state_dict(), f"model_{epoch}.pth")
            print(f"Saved model_{epoch}.pth")

        print("Finished Training")

In [10]:
mydataset = MyAudioDataset('/workspace/small_model_data2', 'recording_01_')

In [11]:
from audiocraft.models import CompressionModel
from audiocraft.models.encodec import InterleaveStereoCompressionModel
model = CompressionModel.get_pretrained('facebook/encodec_32khz')
#model = model.cuda()
comp_model = InterleaveStereoCompressionModel(model)
# move to GPU
##comp_model = comp_model.cuda()

/workspace/venv_work/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [12]:
myTransformer = AudioTransformer(source_comp=model, target_comp=comp_model, d_model=256, nhead=4, num_layers=4, dim_feedforward=1024).cuda()
#myTransformer.train(mydataset, 1, 1, 1)

/workspace/venv_work/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [13]:
myTransformer.train(dataset=mydataset, batch_size=1, epochs=1, lr=0.001)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([1, 4, 1500])


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float